In [24]:
import itertools, json
from IPython.display import clear_output

# dataset for test
# dataset = [
#     {
#         'id': 0,
#         'media': 4,
#         'filmes':[
#             {
#                 'id': 0,
#                 'avaliacao': 4,
#                 'ano': 2010,
#                 'mes': 5,
#                 'dia': 9
#             },
#             {
#                 'id': 2,
#                 'avaliacao': 4,
#                 'ano': 2010,
#                 'mes': 5,
#                 'dia': 9
#             }
#         ],
#         'user_similarity':[
#             [1, 5],
#             [2, 4]
#         ]
#     },
#     {
#         'id': 1,
#         'media': 3.5,
#         'filmes':[
#              {
#                  'id': 1,
#                  'avaliacao': 3,
#                  'ano': 2012,
#                  'mes': 6,
#                  'dia': 12
#              },
#              {
#                  'id': 3,
#                  'avaliacao': 4,
#                  'ano': 2012,
#                  'mes': 6,
#                  'dia': 12
#              }
#         ],
#         'user_similarity':[
#             [0, 4]
#         ]
#     },
#     {
#         'id': 2,
#         'media': 3.7,
#         'filmes':[
#              {
#                  'id': 2,
#                  'avaliacao': 4,
#                  'ano': 2012,
#                  'mes': 6,
#                  'dia': 12
#              },
#              {
#                  'id': 3,
#                  'avaliacao': 3,
#                  'ano': 2012,
#                  'mes': 6,
#                  'dia': 12
#              },
#              {
#                  'id': 4,
#                  'avaliacao': 4,
#                  'ano': 2012,
#                  'mes': 6,
#                  'dia': 12
#              }
#         ],
#         'user_similarity':[
#             [0, 4]
#         ]
#     }
# ]

def find_item_by_id(item_id, data):
    search = list(filter(lambda x: x['id'] == item_id, data))
    if(len(search) == 1):
        return search[0]
    elif(len(search) > 1):
        raise Exception("Not find.")
    
def get_id_users(data):
    return list(map(lambda x: x['id'], data))
    
def get_movies(user_id, data):
    user = find_item_by_id(user_id, data)
    return user['filmes']

def non_watch_movies(user_movies, neighborhood, data):
    all_nb_movies = list(map(lambda x: get_movies(x[0], data), neighborhood)) # all movies seen by the neighborhood
    all_nb_movies = list(itertools.chain.from_iterable(all_nb_movies)) # [[]] -> []
    all_nb_movies = list(set(map(lambda x: x['id'], all_nb_movies))) # IDs of all neighborhood movies
    u_movies = list(map(lambda x: x['id'], user_movies)) # IDs of user movies
    
    return list(filter(lambda x: x not in u_movies, all_nb_movies)) # movies not seen by the user in his neighborhood

def just_famous_movies(neighborhood, movies, n_movies, data): # (neighborhood) nearest neighborhood (e.g. 40 nearest users by similarity)
    nb_ids = list(map(lambda x: x[0], neighborhood)) # id of (id, similarity)
    nb = list(filter(lambda x: x['id'] in nb_ids, data)) # neighbors by id
    filmes_nb = list(map(lambda x: x['filmes'], nb)) # rating of neighbors [[]]
    filmes_nb = list(itertools.chain.from_iterable(filmes_nb)) # [[]] -> []
    
    if len(filmes_nb) == 0:
        return {'famous_movies': []}
    
    filmes_nb = list(map(lambda x: x['id'], filmes_nb)) # IDs of filmes    
    filmes_nb = list(map(lambda x: (x, filmes_nb.count(x)), movies)) # (movie, frequency)
    filmes_nb.sort(key=lambda x: x[1], reverse=True) # sort by highest frequency
    famous_movies = filmes_nb[:n_movies] # n popular movies
    famous_movies = list(map(lambda x: x[0], famous_movies)) # only IDs, frequency is not necessary, just for ranking
    
#     if len(famous_movies) > 0: # hard to happen, but...
        # search neighbors who watched at least one popular movie
    nb_watched_famous_movies = list(filter(lambda x: any(movie in famous_movies for movie in list(map(lambda y: y['id'], x['filmes'])) ), nb))
        
        # return in 'nb' the users obj to ease later, but it can be just IDs
    return {'famous_movies': famous_movies, 'nb': nb_watched_famous_movies}
#     else:
#         print("O valor de n = ", n, " é muito grande. Não existe um filme que ", n, " vizinhos assistiram.")
#         return {}
    
def find_similarity(neighborhood, user_id):
    search = list(filter(lambda x: x[0] == user_id, neighborhood))
    if(len(search) == 1):        
        return search[0][1] # return only similarity value
    else:
        raise Exception('not find')

all_predict = []
unpredictable = []
def predict_user_movie(user, n, min_movies, data):
#     n = 2 # neighborhood
#     min_movies = 1 # min value of filmes
    
    neighborhood = user['user_similarity']
    neighborhood.sort(key=lambda x: x[1], reverse=True) # sort by similarity
    neighborhood = neighborhood[:n] # the real neighborhood
#     print(neighborhood)
    
    # movies watched by neighborhood but not by the user
    non_watched_movies = non_watch_movies(user['filmes'], neighborhood, data)
#     print(non_watched_movies)

    # just the movies with high frequency within neighborhood
    famous_movies = just_famous_movies(neighborhood, non_watched_movies, min_movies, data)
#     print(famous_movies)
    
    if famous_movies['famous_movies'] == []:
        unpredictable.append({'user_id': user['id']})
    else:
        nb = famous_movies['nb'] # users
        famous_movies = famous_movies['famous_movies'] # movies IDs

        nb_ids = list(map(lambda x: x['id'], nb)) # just IDs
        neighborhood = list(filter(lambda x: x[0] in nb_ids, neighborhood)) # filter by IDs -> (user_id, similarity)
    #     print(neighborhood)

        pa_is = [] # final prediction (movie_id, prediction_value)

        for movie_id in famous_movies:
            sum_1 = 0
            sum_2 = 0
            for i in nb:
                try:
                    sim = find_similarity(neighborhood, i['id']) # search in list [(user_id, similarity)]
                    mv = find_item_by_id(movie_id, i['filmes']) # retrieve movie in user movies list
                    sum_1 = sum_1 + ((mv['avaliacao'] - i['media']) * sim)
                    sum_2 = sum_2 + sim
                except:
                    pass # count zero
            if sum_2 > 0:
                pa_i = user['media'] + (sum_1/sum_2)
            else:
                pa_i = user['media']
            pa_is.append({'filme':movie_id, 'predicao': pa_i})

        all_predict.append({'id': user['id'], 'predicoes': pa_is})

dataset = json.load(open('./usuarios.json'))

users = get_id_users(dataset)
# user = find_item_by_id(822109, dataset)
# user
# predict_user_movie(user, 20, 5, dataset)
# print(all_predict)
# print(unpredictable)
tam = len(users)
for id_user in users:
    clear_output()
    print("Restam:", tam-1)
    tam = tam-1
    user = find_item_by_id(id_user, dataset)
    predict_user_movie(user, 20, 5, dataset)

with open('predictable.json', 'w') as outfile:
    json.dump(all_predict, outfile)
    
with open('unpredictable.json', 'w') as outfile:
    json.dump(unpredictable, outfile)

Restam: 0


# Separação dos dados disponíveis em treinamento e testes

In [46]:
from collections import Counter
import json, math
from random import randrange

dataset = json.load(open('./data.json'))

def reservoir_sampling(items, k):
    """ 
    Reservoir sampling algorithm for large sample space or unknow end list
    See <http://en.wikipedia.org/wiki/Reservoir_sampling> for detail>
    Type: ([a] * Int) -> [a]
    Prev constrain: k is positive and items at least of k items
    Post constrain: the length of return array is k
    Credits: https://gist.github.com/m00nlight/bfe54d1b2db362755a3a
    """
    sample = items[0:k]
    
    for i in range(k, len(items)):
        j = randrange(1, i + 1)
        if j < k:
            sample[j] = items[i]

    return sample

filtered = list(filter(lambda x: len(x['filmes']) >= 3, dataset))
# filtered = list(filter(lambda x: len(x['filmes']) >= 24, dataset)) # for tests

trainset = []
testset = []
for item in filtered:
    train = reservoir_sampling(item['filmes'], math.floor(len(item['filmes'])*.75))
    test = list(filter(lambda x: x not in train, item['filmes']))
    trainset.append({'id': item['id'], 'media': item['media'], 'filmes': train})
    testset.append({'id': item['id'], 'media': item['media'], 'filmes': test})

with open('train.json', 'w') as trainfile:
    json.dump(trainset, trainfile)
with open('test.json', 'w') as testfile:
    json.dump(testset, testfile)